In [1]:
import dataclasses
import functools
import os
from collections.abc import Callable

import openai

from effectful.handlers.llm import Template
from effectful.handlers.llm.providers import OpenAIAPIProvider, tool_call
from effectful.handlers.llm.synthesis import ProgramSynthesis
from effectful.ops.semantics import fwd, handler
from effectful.ops.syntax import defop

provider = OpenAIAPIProvider(openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

## Interface

The `robotl.ops.llm` module provides a simplified LLM interface that uses algebraic effects to provide modularity. The module interface consists of:

- A decorator `template` which creates a prompt template from a callable. We should think of the prompt template as an LLM-implemented function with behavior specified by a template string. When a templated function is called, an LLM is invoked to produce the specified behavior. The `__call__` method of a template is a handleable operation.
- An operation `decode` which parses LLM output. `decode(t: type, c: str)` converts an LLM response `c` to the type `t`. It can be handled to provide decoding logic for particular types.
- Interpretations for LLM providers `OpenAIIntp` and callable decoding `ProgramSynthesisIntp`. These interpretations can be composed to handle a variety of template behaviors.

## Prompt Templates

This template function writes (bad) poetry on a given theme. While difficult to implement in Python, an LLM can provide a reasonable implementation.

In [2]:
@Template.define
def limerick(theme: str) -> str:
    """Write a limerick on the theme of {theme}."""
    raise NotImplementedError

If we call the template with a provider interpretation installed, we get reasonable behavior. The LLM is nondeterministic by default, so calling the template twice with the same arguments gives us different results.

Templates are regular callables, so can be converted to operations with `defop` if we want to override the LLM implementation in some cases.

In [3]:
with handler(provider):
    print(limerick("fish"))
    print("-" * 40)
    print(limerick("fish"))

A fish with a wish in the sea,  
Dreamed of climbing a tall, grand tree.  
With scales shining bright,  
It leapt out of sight,  
But decided that swimming's the key.
----------------------------------------
In the pond where the sun loves to glisten,  
A curious fish liked to listen.  
With a flip and a swish,  
It'd grant every wish,  
To the tales that the waters would christen.


If we want deterministic behavior, we can cache the template call.

In [4]:
@functools.cache
@Template.define
def haiku(theme: str) -> str:
    """Write a haiku on the theme of {theme}."""
    raise NotImplementedError


print()
with handler(provider):
    print(haiku("fish"))
    print("-" * 40)
    print(haiku("fish"))


Silver scales shimmer,  
Deep beneath the water's dance—  
Quiet currents hum.
----------------------------------------
Silver scales shimmer,  
Deep beneath the water's dance—  
Quiet currents hum.


## Converting LLM Results to Python Objects

Type conversion is handled by `decode`. By default, primitive types are converted. `DecodeError` is raised if a response cannot be converted.

In [5]:
@Template.define
def primes(first_digit: int) -> int:
    """Give a prime number with {first_digit} as the first digit."""
    raise NotImplementedError


with handler(provider):
    assert type(primes(6)) is int

More complex types can be converted by providing handlers for `decode`. `ProgramSynthesisIntp` provides a `decode` handler that parses Python callables.

In [6]:
@Template.define
def count_char(char: str) -> Callable[[str], int]:
    """Write a function which takes a string and counts the occurrances of '{char}'."""
    raise NotImplementedError


with handler(provider), handler(ProgramSynthesis()):
    count_a = count_char("a")
    assert callable(count_a)
    assert count_a("banana") == 3
    assert count_a("cherry") == 0


## Tool Calling

Passing `Operation`s to `Template.define` makes them available for the LLM to call as tools. The description of these operations is inferred from their type annotations and docstrings.

Tool calls are mediated by a helper operation `tool_call`. Handling this operation allows tool use to be tracked or logged.

In [7]:
@defop
def cities() -> list[str]:
    return ["Chicago", "New York", "Barcelona"]


@defop
def weather(city: str) -> str:
    status = {"Chicago": "cold", "New York": "wet", "Barcelona": "sunny"}
    return status.get(city, "unknown")


@Template.define(tools=[cities, weather])
def vacation() -> str:
    """Use the provided tools to suggest a city that has good weather."""
    raise NotImplementedError


def log_tool_call(_, tool, *args, **kwargs):
    result = fwd()
    print(f"Tool call: {tool}(*{args}, **{kwargs}) -> {result}")
    return result


with handler(provider), handler({tool_call: log_tool_call}):
    print(vacation())

Tool call: cities(*(), **{}) -> ['Chicago', 'New York', 'Barcelona']
Tool call: weather(*(), **{'city': 'Chicago'}) -> cold
Tool call: weather(*(), **{'city': 'New York'}) -> wet
Tool call: weather(*(), **{'city': 'Barcelona'}) -> sunny
Based on the current weather conditions, Barcelona has sunny weather and would be a good choice!


## Structured Output Generation

Constrained generation is used for any type that is convertible to a Pydantic model.

In [8]:
@dataclasses.dataclass
class KnockKnockJoke:
    whos_there: str
    punchline: str


@Template.define
def write_joke(theme: str) -> KnockKnockJoke:
    """Write a knock-knock joke on the theme of {theme}."""
    raise NotImplementedError


@Template.define
def rate_joke(joke: KnockKnockJoke) -> bool:
    """Decide if {joke} is funny or not"""
    raise NotImplementedError


def do_comedy():
    joke = write_joke("lizards")
    print("> You are onstage at a comedy club. You tell the following joke:")
    print(
        f"Knock knock.\nWho's there?\n{joke.whos_there}.\n{joke.whos_there} who?\n{joke.punchline}"
    )
    if rate_joke(joke):
        print("> The crowd laughs politely.")
    else:
        print("> The crowd stares in stony silence.")


with handler(provider):
    do_comedy()

> You are onstage at a comedy club. You tell the following joke:
Knock knock.
Who's there?
Iguana.
Iguana who?
Iguana come inside, it's too cold out here!
> The crowd laughs politely.
